In [1]:
import cv2
import math
import argparse
from keras.models import model_from_json
from keras.preprocessing import image
import numpy as np


In [2]:
faceProto="models/opencv_face_detector.pbtxt"
faceModel="models/opencv_face_detector_uint8.pb"
genderProto="models/gender_deploy.prototxt" # Describes the Network Configuration
genderModel="models/gender_net.caffemodel" # Defines the Internal States of the parameters of the Layers
ageNet=cv2.dnn.readNet("models/age_net.caffemodel","models/age_deploy.prototxt")
emotionNet = model_from_json(open("models/fer.json", "r").read())
emotionNet.load_weights('models/model_filter.h5')
faceNet=cv2.dnn.readNet(faceModel,faceProto) 
genderNet=cv2.dnn.readNet(genderModel,genderProto)

In [3]:
def detectFaces(net, frame, conf_threshold=0.7):
    frame1=frame.copy()
    frameHeight=frame1.shape[0]
    frameWidth=frame1.shape[1]
    blob=cv2.dnn.blobFromImage(frame1, 1.0, (300, 300), [104, 117, 123], True, False) 
    
    # [blobFromImage] creates 4-dimensional blob from image. 
    # Optionally resizes and crops image from center, subtract mean values, scales values by scalefactor, swap Blue and Red channels.


    # set the input to the pre-trained deep learning network and obtain the output predicted probabilities 

    net.setInput(blob) #Passing the blob through the network 
    detections=net.forward() #Grabbing the Detections/Predictions
    faceBoxes=[]

    # Filter out weak detections by ensuring the confidence is greater than the minimum confidence
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>conf_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)

            faceBoxes.append([x1,y1,x2,y2]) #Bounding Box Co-ordinates

            cv2.rectangle(frame1, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    return frame1,faceBoxes

    #cv2.dnn.blobFromImage function returns a blob which is the input image after mean subtraction, normalizing, and channel swapping.

In [4]:
def detectEmotion(net,frame,faceBoxes):
    gray_img= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    for faceBox in faceBoxes:
        x=faceBox[0]
        y=faceBox[1]
        w=np.absolute(faceBox[2]-faceBox[0])
        h=np.absolute(faceBox[1]-faceBox[3])
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels=img_pixels.astype(float)
        img_pixels /= 255
        emotionPreds = emotionNet.predict(img_pixels)
        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        emotion=emotions[emotionPreds[0].argmax()] #Disp
    return emotion

In [5]:
MODEL_MEAN_VALUES=(78.4263377603, 87.7689143744, 114.895847746) #Initializing the Mean Values for the Model
genderList=['Male','Female']
ageList=['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']

In [6]:
def detectGenderAge(net,aNet,frame, faceBoxes):
    for faceBox in faceBoxes:
        face=frame[max(0,faceBox[1]-padding):
                   min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding)
                   :min(faceBox[2]+padding, frame.shape[1]-1)]

        blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=True) 

        net.setInput(blob) #Passing the blob through the Neural Net
        genderPreds=net.forward() #Grabbing the Detections/Predictions
        gender=genderList[genderPreds[0].argmax()] #Displaying the Top Prediction
        #The function putText renders the specified text string in the image.
        aNet.setInput(blob)
        agePreds=aNet.forward()
        age=ageList[agePreds[0].argmax()]
        emotion1=detectEmotion(emotionNet,frame,faceBoxes)
        cv2.putText(frame, f'{gender},{age},{emotion1}', (faceBox[0], faceBox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2, cv2.LINE_AA)
    
    return frame

In [19]:
img = 'girl2.jpg'

In [7]:
cap=cv2.VideoCapture(0)
padding=20

# while cv2.waitKey(1)<0:
while True:
    hasFrame,frame=cap.read()
    if not hasFrame:
        cv2.waitKey()
        break

    resultImg,faceBoxes=detectFaces(faceNet,frame)
    if not faceBoxes:
        print('No faces found....')

    resultImg1 = detectGenderAge(genderNet,ageNet, resultImg, faceBoxes)
    cv2.imshow("Gender Detection", resultImg1) #Display the Image in an OpenCV Window

    if cv2.waitKey(1) == ord('q'):
    #if key >= 0:
        break

cap.release()
cv2.destroyAllWindows()

No faces found....
